In [1]:
from utils import build_url, load_json, save_json, call_with_retry, save_execution_state, load_execution_state
import os, shutil

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

with open(API_KEY_FILE) as f:
    api_key = f.read()

execution_cache_dir = 'current_execution'
games_ids_state = f'{execution_cache_dir}/games.pkl'
accounts_state = f'{execution_cache_dir}/accounts.pkl'

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 20/100 # esperar 20ms entre peticiones
delay = MIN_DELAY

headers = {
  'X-Riot-Token': api_key
}

games = load_json('outputs/games_ids.json')
print(games[0:20])

[5287968773, 5310382098, 5270077462, 5294719002, 5226627099, 5311365148, 5303435290, 5284659231, 5296980005, 5302550566, 5282594856, 5280006188, 5301501996, 5310840877, 5180227632, 5250547765, 5270306870, 5112430654, 5239767106, 5213421638]


In [2]:
endpoint = '/lol/match/v4/matches/{}'
output_dir = 'games/'

for game in games[0:100]:
    full_url = build_url(REGION, BASE_URL, endpoint, game)
    response = call_with_retry(full_url, headers)
    save_json(response, f'{output_dir}{game}.json')
    print(f'Guardada partida {game}')

Guardada partida 5287968773
Guardada partida 5310382098
Guardada partida 5270077462
Guardada partida 5294719002
Guardada partida 5226627099
Guardada partida 5311365148
Guardada partida 5303435290
Guardada partida 5284659231
Guardada partida 5296980005
Guardada partida 5302550566
Guardada partida 5282594856
Guardada partida 5280006188
Guardada partida 5301501996
Guardada partida 5310840877
Guardada partida 5180227632
Guardada partida 5250547765
Guardada partida 5270306870
Guardada partida 5112430654
Guardada partida 5239767106
Guardada partida 5213421638
Guardada partida 5223415880
Guardada partida 5286494285
Guardada partida 5290295380
Guardada partida 5284560985
Guardada partida 5214077020
Guardada partida 5212668005
Guardada partida 5237506150
Guardada partida 5292064871
Guardada partida 5298356330
Guardada partida 5208244331
Guardada partida 5273026667
Guardada partida 5256642675
Guardada partida 5278793844
Guardada partida 5247238261
Guardada partida 5255168121
Guardada partida 531

In [3]:
from collections import defaultdict

champions_item_lists = defaultdict(list)
games_dir = 'outputs/games'

for game_file in os.listdir(games_dir):
    if game_file.endswith('.json'):
        game = load_json(f'{games_dir}/{game_file}')
        for player in game['participants']:
            items = [value for stat, value in player['stats'].items() if 'item' in stat and value != 0]
            champions_item_lists[player['championId']].append(items)

save_json(champions_item_lists, 'champion_item_lists.json')

In [4]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)